### SVD and Pseudo inverse with Julia
The purpose of this notebook is to show different ways of calculating the inverse of a matrix for use with solving a least squares problems.

In [1]:
J = [1 2 3 4
     3 4 5 6
     3.1 4.1 5.1 6.2]

3×4 Array{Float64,2}:
 1.0  2.0  3.0  4.0
 3.0  4.0  5.0  6.0
 3.1  4.1  5.1  6.2

First work out the svd using Julia inbuilt functions.

In [2]:
u, σ, v = svd(J)
display(u)
display(σ)
display(v)

3×3 Array{Float64,2}:
 -0.377181  -0.926135  -0.00284064
 -0.645797   0.265206  -0.715969  
 -0.663838   0.268216   0.698126  

3-element Array{Float64,1}:
 14.3552   
  0.892362 
  0.0382432

4×3 Array{Float64,2}:
 -0.304591   0.7855      0.351494
 -0.422096   0.345416   -0.189356
 -0.539602  -0.0946675  -0.730206
 -0.661732  -0.504694    0.554432

Now recombine the components of the svd, in order to get the original matrix J.
Note that the Julia svd returns v rather than v' (transpose), which is what numpy does.

In [3]:
#recombine for form the original matrix.
#remeber the Julia svd doesn't return v' just v
Σ = Diagonal(σ)
J_1 = u * Σ * v'

3×4 Array{Float64,2}:
 1.0  2.0  3.0  4.0
 3.0  4.0  5.0  6.0
 3.1  4.1  5.1  6.2

### Pseudo Inverse
We now look at different ways of calculating the Pseudo Inverse.
* pinv built in Julia function without a tolerance
* manually calculate the pinv, by inverting Sigma without any tolerance
* manually calculate the pinv, by inverting Sigma using a tolerance to work out the rank
* pinv built in Julia fuction with a tolerance

Using pinv in Julia without a tolerance

In [4]:
pseudoInverse = pinv(J)

4×3 Array{Float64,2}:
 -0.833333   -6.33333    6.66667
 -0.333333    3.66667   -3.33333
  0.166667   13.6667   -13.3333 
  0.5       -10.5       10.0    

Manual calculation inverting Sigma without a tolerance

In [5]:
display(Σ)
m, n = size(Σ)
Σ_inv = zeros(Σ, Float64)
tol = 0.0027
for i = 1:m, j = 1:m
    if i==j && Σ[i,j] >= tol
        Σ_inv[i,j] = 1/Σ[i,j]
    end
end
display(Σ_inv)
J_inv = v * Σ_inv * u'

3×3 Diagonal{Float64}:
 14.3552   ⋅         ⋅       
   ⋅      0.892362   ⋅       
   ⋅       ⋅        0.0382432

3×3 Diagonal{Float64}:
 0.0696611   ⋅         ⋅    
  ⋅         1.12062    ⋅    
  ⋅          ⋅       26.1484

4×3 Array{Float64,2}:
 -0.833333   -6.33333    6.66667
 -0.333333    3.66667   -3.33333
  0.166667   13.6667   -13.3333 
  0.5       -10.5       10.0    

Manually work out the inverse of Sigma using a tolerance

In [6]:
#work out the rank of the 'singular values'
rank = sum(σ.> tol * maximum(σ))
σ_rank_inv = [ 1/x for x in σ[1:rank]]
Σ_rank_inv = Diagonal(σ_rank_inv)
J_inv = v[:,1:rank] * Σ_rank_inv * u'[1:rank,:]

4×3 Array{Float64,2}:
 -0.807225   0.247149     0.250181  
 -0.347398   0.121645     0.12334   
  0.112428  -0.00385969  -0.00350087
  0.541182  -0.120223    -0.121094  

Julia pinv using a tolerance

In [7]:
pseudoInverse = pinv(J, tol)

4×3 Array{Float64,2}:
 -0.807225   0.247149     0.250181  
 -0.347398   0.121645     0.12334   
  0.112428  -0.00385969  -0.00350087
  0.541182  -0.120223    -0.121094  

### Using Eigendecomposition
We want to find the solution to $A \hat{x} = b $

$$ A^T A \hat{x} = A^T b $$
$$ \hat{x} = (A^T A)^{-1} A^T b $$

We want to find $(A^T A)^{-1} A^T$, which we'll call the pseudo inverse.

In order to find $(A^T A)^{-1}$ we use the eigendecomposition.

$(A^T A)^{-1} = U \Sigma^{-1} U^T$

where

$U$ = eigenvectors of $A^T A$

$\Sigma$ = eigenvaluues of $A^T A$

Note that the eigenvalues of $A^T A$ are the square of the eigen values of $A$

In [8]:
λ, V = eig(J'*J)
display(λ)
display(V)

4-element Array{Float64,1}:
   1.11022e-16
   0.00146255 
   0.79631    
 206.072      

4×4 Array{Float64,2}:
  0.408248  -0.351494  -0.7855     0.304591
 -0.816497   0.189356  -0.345416   0.422096
  0.408248   0.730206   0.0946675  0.539602
  0.0       -0.554432   0.504694   0.661732

In [9]:
λ |> sort

4-element Array{Float64,1}:
   1.11022e-16
   0.00146255 
   0.79631    
 206.072      

In [10]:
svdvals(J).^2 |> sort

3-element Array{Float64,1}:
   0.00146255
   0.79631   
 206.072     

In [11]:
eigvals(J'*J).^0.5

4-element Array{Float64,1}:
  1.05367e-8
  0.0382432 
  0.892362  
 14.3552    

In [12]:
V2 = eigvecs(J*J')

3×3 Array{Float64,2}:
 -0.00284064   0.926135  -0.377181
 -0.715969    -0.265206  -0.645797
  0.698126    -0.268216  -0.663838

In [13]:
λ_rank = λ[2:end]

3-element Array{Float64,1}:
   0.00146255
   0.79631   
 206.072     

In [14]:
V_rank = V[:,2:end]

4×3 Array{Float64,2}:
 -0.351494  -0.7855     0.304591
  0.189356  -0.345416   0.422096
  0.730206   0.0946675  0.539602
 -0.554432   0.504694   0.661732

Recombine the J'*J from the the eigen decomposition

In [15]:
J_TxJ = V_rank * Diagonal(λ_rank) * V_rank'

4×4 Array{Float64,2}:
 19.61  26.71  33.81  41.22
 26.71  36.81  46.91  57.42
 33.81  46.91  60.01  73.62
 41.22  57.42  73.62  90.44

Now work out the the inverse of J using the eigen decomposition

In [16]:
function invertEigenValues(λ)
    λ_inv = [(abs(x/λ[end]))^0.5 >=tol ? 1/x : 0 for x in λ]
    Diagonal(λ_inv)
end;

In [17]:
λ_inv = invertEigenValues(λ)
J_inv = V * λ_inv * V' * J'

4×3 Array{Real,2}:
 -0.807225   0.247149     0.250181  
 -0.347398   0.121645     0.12334   
  0.112428  -0.00385969  -0.00350087
  0.541182  -0.120223    -0.121094  

In [18]:
a, b, c = svd(J'*J)
display(a)
display(b)
display(c)

4×4 Array{Float64,2}:
 -0.304591  -0.7855      0.351494  -0.408248   
 -0.422096  -0.345416   -0.189356   0.816497   
 -0.539602   0.0946675  -0.730206  -0.408248   
 -0.661732   0.504694    0.554432  -9.49768e-13

4-element Array{Float64,1}:
 206.072     
   0.79631   
   0.00146255
   4.9016e-16

4×4 Array{Float64,2}:
 -0.304591  -0.7855      0.351494   0.408248 
 -0.422096  -0.345416   -0.189356  -0.816497 
 -0.539602   0.0946675  -0.730206   0.408248 
 -0.661732   0.504694    0.554432   1.254e-13

In [19]:
vecs = eigvecs(J*J')

3×3 Array{Float64,2}:
 -0.00284064   0.926135  -0.377181
 -0.715969    -0.265206  -0.645797
  0.698126    -0.268216  -0.663838

In [20]:
u, σ, v = svd(J)
display(u)
display(σ)
display(v)

3×3 Array{Float64,2}:
 -0.377181  -0.926135  -0.00284064
 -0.645797   0.265206  -0.715969  
 -0.663838   0.268216   0.698126  

3-element Array{Float64,1}:
 14.3552   
  0.892362 
  0.0382432

4×3 Array{Float64,2}:
 -0.304591   0.7855      0.351494
 -0.422096   0.345416   -0.189356
 -0.539602  -0.0946675  -0.730206
 -0.661732  -0.504694    0.554432

In [21]:
#eigen values are the squares of the singular values (from the svd)
λ, V = eig(J'*J)
display(λ)
display(V)

4-element Array{Float64,1}:
   1.11022e-16
   0.00146255 
   0.79631    
 206.072      

4×4 Array{Float64,2}:
  0.408248  -0.351494  -0.7855     0.304591
 -0.816497   0.189356  -0.345416   0.422096
  0.408248   0.730206   0.0946675  0.539602
  0.0       -0.554432   0.504694   0.661732

In [22]:
V'

4×4 Array{Float64,2}:
  0.408248  -0.816497  0.408248    0.0     
 -0.351494   0.189356  0.730206   -0.554432
 -0.7855    -0.345416  0.0946675   0.504694
  0.304591   0.422096  0.539602    0.661732

In [23]:
J'*J

4×4 Array{Float64,2}:
 19.61  26.71  33.81  41.22
 26.71  36.81  46.91  57.42
 33.81  46.91  60.01  73.62
 41.22  57.42  73.62  90.44

In [24]:
#eigen value decomposition to reform J'*J
V*Diagonal(λ)*V' # = J'*J

4×4 Array{Float64,2}:
 19.61  26.71  33.81  41.22
 26.71  36.81  46.91  57.42
 33.81  46.91  60.01  73.62
 41.22  57.42  73.62  90.44

See the following article.
https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix

Point to note is that the inverse of V, i.e. the eigen vectors of J'*J is the same as taking the transpose of V, i.e. V'.  Generally is the transpose of a symmetric matrix equal to its inverse?

In [25]:
inv(V)

4×4 Array{Float64,2}:
  0.408248  -0.816497  0.408248   -3.67335e-17
 -0.351494   0.189356  0.730206   -0.554432   
 -0.7855    -0.345416  0.0946675   0.504694   
  0.304591   0.422096  0.539602    0.661732   

In [26]:
V'

4×4 Array{Float64,2}:
  0.408248  -0.816497  0.408248    0.0     
 -0.351494   0.189356  0.730206   -0.554432
 -0.7855    -0.345416  0.0946675   0.504694
  0.304591   0.422096  0.539602    0.661732

In [27]:
V

4×4 Array{Float64,2}:
  0.408248  -0.351494  -0.7855     0.304591
 -0.816497   0.189356  -0.345416   0.422096
  0.408248   0.730206   0.0946675  0.539602
  0.0       -0.554432   0.504694   0.661732

In [28]:
#Some other stuff with matrices
V

4×4 Array{Float64,2}:
  0.408248  -0.351494  -0.7855     0.304591
 -0.816497   0.189356  -0.345416   0.422096
  0.408248   0.730206   0.0946675  0.539602
  0.0       -0.554432   0.504694   0.661732

In [29]:
#2nd row
V[2,:]
#3rd column
V[:,3]
#columns 1 to 3
V[:,1:3]
#columns 2 to end
V[:,2:end]
#reverse columns 2 to end
V[:,2:-1:1]

4×2 Array{Float64,2}:
 -0.351494   0.408248
  0.189356  -0.816497
  0.730206   0.408248
 -0.554432   0.0     

In [30]:
b = rand(5,1)
A = rand(5,5)
x = inv(A)*b
norm(A*x-b)  # check residual

4.710277376051325e-16